In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import layers
import keras

%matplotlib inline
tf.random.set_seed(42)

2024-01-11 09:50:57.134943: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-11 09:50:57.163170: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-11 09:50:57.320962: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-11 09:50:57.322201: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-11 09:50:58.341691: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
import numpy as np
import pandas as pd
import math
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
#import plotly.graph_objects as go
import tensorflow as tf
#import tensorflow_addons as tfa

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVR
#from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from keras import Sequential
from keras import layers
from keras.models import Model
from keras.layers import LSTM, BatchNormalization, Dropout, Dense, Flatten, Conv1D
from keras.layers import MaxPooling1D, GRU, Input,Masking, Concatenate, dot
from keras.optimizers import Adam, SGD
from keras.losses import MeanAbsoluteError
from keras.metrics import RootMeanSquaredError
from keras.callbacks import EarlyStopping
from keras.callbacks import LearningRateScheduler
from sklearn.metrics import mean_squared_error

In [3]:
df = pd.read_excel(r"~/Downloads/BID_RA_data/BID_RA_19_20.xlsx")

In [4]:
df.dtypes

Order ID                                  int64
Order Date                       datetime64[ns]
FY_OrderDate                             object
Month_Name_OrderDate                     object
Purchase Mode                            object
Bid start date                   datetime64[ns]
Bid end date                     datetime64[ns]
HOD ID                                   object
Seller ID                                object
Order_Quarter                            object
Order MSME Verified                      object
MSME verfied gender wise                 object
MSME verified social category            object
Startup verfied                          object
Sub Category ID                          object
Expected delivery date           datetime64[ns]
Shipment received date                   object
Order Qty                                 int64
Order value                             float64
Consignment ID                            int64
GeM Invoice Number                      

In [5]:
df1 = df.drop(df.columns[[0, 7, 19, 20, 21, 31]], axis=1)

In [6]:
df1["price"] = df1["Order value"]/df1["Order Qty"]

In [7]:
import datetime
from datetime import datetime as dt

In [8]:
df1["Time taken to deliver"] = (df1["Expected delivery date"] - df1["Order Date"]).dt.days

In [9]:
df1.drop(columns=["Month_Name_OrderDate", "Order_Quarter", "Invoice Date","CRAC date", "Payment Date"], inplace=True)

In [10]:
columns_to_encode = ["Purchase Mode", "MSME verfied gender wise", "Order MSME Verified", "Startup verfied"]

In [11]:
le = LabelEncoder()

In [12]:
for column in columns_to_encode:
       df1[column+'_encoded'] = le.fit_transform(df1[column])

In [13]:
df1 = df1.drop(columns=["Purchase Mode", "MSME verfied gender wise", "Order MSME Verified", "Startup verfied"])

In [14]:
df1 = df1.drop(columns=["FY_OrderDate", "Bid start date", "Bid end date"])

In [15]:
df1["Payment Type"+'_encoded'] = le.fit_transform(df1["Payment Type"])

In [16]:
df1["MSME verified social category"+'_encoded'] = le.fit_transform(df1["MSME verified social category"])

In [17]:
df1 = df1.drop(columns=["MSME verified social category", "Payment Type"])

In [18]:
df1 = df1.drop(columns=["Order Date", "Expected delivery date", "Shipment received date"])

In [19]:
df1 = df1.drop(columns=['Seller ID'])

In [20]:
df1["Sub Category ID"].value_counts()

Sub Category ID
home_info_comp_comp_de52008682          811482
home_comm_nonm_peda_bicy                 89195
home_comm_moto_auto_ambu                 61019
home_appa_foot_sh88838715_sh71256827      8535
home_furn_acco_furn_almi                  7739
                                         ...  
home_mine_fabr_spec_cott                     1
home_medi_medi_me16126272_me82121660         1
home_offi_of64586874_draf_geom               1
home_appa_pers_bath_swac                     1
home_info_data_netw_outd                     1
Name: count, Length: 1352, dtype: int64

In [ ]:
home_info_comp_comp_de52008682

In [21]:
df2=df1[df1["Sub Category ID"]=="home_info_comp_comp_de52008682"]

In [22]:
df2.shape

(811482, 16)

In [23]:
df2 = df2.drop(columns=['Sub Category ID'])

In [24]:
X = df2.drop(columns=['Time taken to deliver'])
y = df2['Time taken to deliver']

In [25]:
X.columns

Index(['Order Qty', 'Order value', 'Shipment Quantity Received',
       'Shipment Quantity Rejected', 'Shipment Quantity Accepted',
       'CRAC amount', 'Total payments', 'price', 'Purchase Mode_encoded',
       'MSME verfied gender wise_encoded', 'Order MSME Verified_encoded',
       'Startup verfied_encoded', 'Payment Type_encoded',
       'MSME verified social category_encoded'],
      dtype='object')

In [26]:
X.shape

(811482, 14)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [28]:
X_train.head()

,Order Qty,Order value,Shipment Quantity Received,Shipment Quantity Rejected,Shipment Quantity Accepted,CRAC amount,Total payments,price,Purchase Mode_encoded,MSME verfied gender wise_encoded,Order MSME Verified_encoded,Startup verfied_encoded,Payment Type_encoded,MSME verified social category_encoded
604980,1432,74870688.0,2,0,2,104568.0,52284.0,52284.0,0,0,1,0,1,0
393838,953,48015952.0,2,0,2,100768.0,90691.0,50384.0,1,0,1,0,1,0
906272,1432,74870688.0,1,0,1,52284.0,104568.0,52284.0,0,0,1,0,1,0
559813,1432,74870688.0,2,0,2,104568.0,52284.0,52284.0,0,0,1,0,1,0
1043661,1432,74870688.0,2,0,2,104568.0,104568.0,52284.0,0,0,1,0,1,0


In [29]:
X_train_bil, y_train_bil = np.array(X_train), np.array(y_train)
X_test_bil, y_test_bil = np.array(X_test), np.array(y_test)
#X_train_bil = X_train_bil.reshape((X_train_bil.shape[0], 1, X_train_bil.shape[1]))
#X_test_bil = X_test_bil.reshape((X_test_bil.shape[0], 1, X_test_bil.shape[1]))
print(X_train_bil.shape, y_train_bil.shape, X_test_bil.shape, y_test_bil.shape)

(649185, 14) (649185,) (162297, 14) (162297,)


In [30]:
from keras.callbacks import EarlyStopping

In [135]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,Reshape
from keras.layers import Conv1D, MaxPooling1D, LeakyReLU
import np_utils
from keras.layers import GRU
model_GRU = Sequential()

model_GRU.add(GRU(units=50, input_shape=(14,1),return_sequences=False))
model_GRU.add(Activation('tanh'))
model_GRU.add(Dropout(0.3))
model_GRU.add(Dense(1))
model_GRU.add(Activation('relu'))
model_GRU.compile(loss='mse', optimizer='adam', metrics = ('MAPE'))

In [136]:
model_GRU.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_4 (GRU)                 (None, 50)                7950      
                                                                 
 activation_8 (Activation)   (None, 50)                0         
                                                                 
 dropout_4 (Dropout)         (None, 50)                0         
                                                                 
 dense_4 (Dense)             (None, 1)                 51        
                                                                 
 activation_9 (Activation)   (None, 1)                 0         
                                                                 
Total params: 8001 (31.25 KB)
Trainable params: 8001 (31.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [137]:
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=60,verbose=1)]

In [138]:
history_GRU = model_GRU.fit(X_train_bil, y_train_bil, validation_data=(X_test_bil, y_test_bil), batch_size = 128, epochs = 60, callbacks=callbacks)

Epoch 1/60
5072/5072 [==============================] - 36s 7ms/step - loss: 55.6374 - MAPE: 10769.9736 - val_loss: 1.6808 - val_MAPE: 1.4891
Epoch 2/60
5072/5072 [==============================] - 33s 6ms/step - loss: 8.0559 - MAPE: 36992.1055 - val_loss: 0.5216 - val_MAPE: 1.0386
Epoch 3/60
5072/5072 [==============================] - 31s 6ms/step - loss: 6.6016 - MAPE: 37358.1523 - val_loss: 0.5223 - val_MAPE: 1.2131
Epoch 4/60
5072/5072 [==============================] - 32s 6ms/step - loss: 5.3271 - MAPE: 38917.6133 - val_loss: 0.4228 - val_MAPE: 0.6301
Epoch 5/60
5072/5072 [==============================] - 34s 7ms/step - loss: 4.2015 - MAPE: 38103.2852 - val_loss: 0.4853 - val_MAPE: 1.1517
Epoch 6/60
5072/5072 [==============================] - 31s 6ms/step - loss: 3.2289 - MAPE: 34017.4180 - val_loss: 0.4145 - val_MAPE: 0.6727
Epoch 7/60
5072/5072 [==============================] - 34s 7ms/step - loss: 2.3901 - MAPE: 38449.8242 - val_loss: 0.3912 - val_MAPE: 0.8350
Epoch 8/60
5

In [132]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

In [140]:
pred_GRU = model_GRU.predict(X_test_bil)

5072/5072 [==============================] - 8s 2ms/step


In [31]:
from sklearn.metrics import mean_absolute_error
from keras.layers import Bidirectional
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

In [142]:
mse_gru_seen = mean_squared_error(y_test, pred_GRU)
rmse_gru_seen = np.sqrt(mse_gru_seen)
print(f"Root Mean Squared Error: {rmse_gru_seen}")
print(f"Mean Squared Error: {mse_gru_seen}")

Root Mean Squared Error: 0.45439463453759427
Mean Squared Error: 0.20647448389655387


In [143]:
# Create a DataFrame of Real and Predicted values
prices_gru_seen = pd.DataFrame({
    "Real": y_test.ravel(),
    "Predicted": pred_GRU.ravel()
}, index = df1.index[-len(y_test): ]) 
prices_gru_seen.head()

,Real,Predicted
886268,30,30.025566
886269,30,30.038054
886270,30,30.025566
886271,30,30.025566
886272,30,30.038054


In [63]:
from catboost import CatBoostRegressor
model_cb = CatBoostRegressor(iterations=1000,  # Adjust the number of iterations as needed
                             learning_rate=0.01,  # Adjust the learning rate as needed
                             depth=10,  # Adjust the depth of the trees
                             loss_function='RMSE',  # Specify the loss function for regression
                             random_seed=42,
                            verbose=100)
model_cb.fit(X_train_bil, y_train_bil)
y_pred_cb = model_cb.predict(X_test_bil)
from sklearn.metrics import mean_squared_error
mse_cat = mean_squared_error(y_test_bil, y_pred_cb)
rmse_cat = np.sqrt(mse_cat)
print(f"Root Mean Squared Error: {rmse_cat}")
print(f"Mean Squared Error: {mse_cat}")

0:	learn: 4.3064231	total: 45.5ms	remaining: 45.4s
100:	learn: 1.6309289	total: 5.56s	remaining: 49.5s
200:	learn: 0.7059445	total: 11.3s	remaining: 44.9s
300:	learn: 0.4372984	total: 16.8s	remaining: 39.1s
400:	learn: 0.3711044	total: 22.4s	remaining: 33.4s
500:	learn: 0.3472348	total: 28s	remaining: 27.9s
600:	learn: 0.3317389	total: 33.6s	remaining: 22.3s
700:	learn: 0.3189981	total: 39.3s	remaining: 16.8s
800:	learn: 0.3057439	total: 45s	remaining: 11.2s
900:	learn: 0.2951812	total: 50.8s	remaining: 5.58s
999:	learn: 0.2859277	total: 56.7s	remaining: 0us
Root Mean Squared Error: 0.3793144059783939
Mean Squared Error: 0.1438794185827418


In [65]:
# Create a DataFrame of Real and Predicted values
prices_catb_seen = pd.DataFrame({
    "Real": y_test.ravel(),
    "Predicted": y_pred_cb.ravel()
}, index = df1.index[-len(y_test): ]) 
prices_catb_seen.head()

,Real,Predicted
886268,30,30.001597
886269,30,30.003218
886270,28,27.999835
886271,30,30.002701
886272,30,30.001988


In [ ]:
## On unseeen data

In [34]:
df22 = pd.read_excel(r"~/Downloads/BID_RA_data/BID_RA_22_23.xlsx")

In [35]:
df22_1 = df22.drop(df22.columns[[0, 7, 19, 20, 21, 31 ]], axis=1)
df22_1["price"] = df22_1["Order value"]/df22_1["Order Qty"]
df22_1["Time taken to deliver"] = (df22_1["Expected delivery date"] - df22_1["Order Date"]).dt.days
df22_1.drop(columns=["Month_Name_OrderDate", "Order_Quarter", "Invoice Date","CRAC date", "Payment Date"], inplace=True)
for column in columns_to_encode:
       df22_1[column+'_encoded'] = le.fit_transform(df22_1[column])
df22_1 = df22_1.drop(columns=["FY_OrderDate", "Bid start date", "Bid end date"])
df22_1["Payment Type"+'_encoded'] = le.fit_transform(df22_1["Payment Type"])
df22_1["MSME verified social category"+'_encoded'] = le.fit_transform(df22_1["MSME verified social category"])

df22_1= df22_1.drop(columns=['Seller ID'])
df22_1 = df22_1.drop(columns=["MSME verified social category", "Payment Type"])
df22_1 = df22_1.drop(columns=["Order Date", "Expected delivery date", "Shipment received date"])
df22_1 = df22_1.drop(columns=["Purchase Mode", "Order MSME Verified", "MSME verfied gender wise", "Startup verfied"])
df22_1["Sub Category ID"].value_counts()
#df22_1=df22_1[df22_1["Sub Category ID"]=="home_comm_nonm_peda_bicy"]
#print(df22_1.shape)
#df22_1 = df22_1.drop(columns=["Sub Category ID"])
#I = df22_1.drop(columns=['Time taken to deliver'])
#j = df22_1['Time taken to deliver']
#I_train, I_test, j_train, j_test = train_test_split(I, j, test_size=0.2)
#I_train_bil, j_train_bil = np.array(I_train), np.array(j_train)
#I_test_bil, j_test_bil = np.array(I_test), np.array(j_test)

#print(I_train_bil.shape, j_train_bil.shape, I_test_bil.shape, j_test_bil.shape)

Sub Category ID
home_info_comp_comp_de52008682          305420
home_info_comp_co36451542_inte          172531
home_elec_elec_powe_line                170724
home_elec_elec_powe_onli                170494
home_univ_univ_univ_univ                133236
                                         ...  
home_powe_elec_elec_weld                     1
home_dome_cons_audi_di68048835               1
home_dist_he28016675_vent_smok               1
home_labo_clin_clin_ferr                     1
home_dome_do42264188_dome_do10278707         1
Name: count, Length: 1380, dtype: int64

In [36]:
#df33=df22_1[df22_1["Sub Category ID"]=="home_comm_nonm_peda_bicy"]
df33=df22_1[df22_1["Sub Category ID"]=="home_info_comp_comp_de52008682"]


In [37]:
df33.shape

(305420, 16)

In [38]:
df33.columns

Index(['Sub Category ID', 'Order Qty', 'Order value',
       'Shipment Quantity Received', 'Shipment Quantity Rejected',
       'Shipment Quantity Accepted', 'CRAC amount', 'Total payments', 'price',
       'Time taken to deliver', 'Purchase Mode_encoded',
       'MSME verfied gender wise_encoded', 'Order MSME Verified_encoded',
       'Startup verfied_encoded', 'Payment Type_encoded',
       'MSME verified social category_encoded'],
      dtype='object')

In [39]:
df33 = df33.drop(columns=["Sub Category ID"])
I = df33.drop(columns=['Time taken to deliver'])
j = df33['Time taken to deliver']
I_train, I_test, j_train, j_test = train_test_split(I, j, test_size=0.2)
I_train_bil, j_train_bil = np.array(I_train), np.array(j_train)
I_test_bil, j_test_bil = np.array(I_test), np.array(j_test)

print(I_train_bil.shape, j_train_bil.shape, I_test_bil.shape, j_test_bil.shape)

(244336, 14) (244336,) (61084, 14) (61084,)


In [67]:
I_test_bil

array([[2.49200000e+03, 1.09530876e+08, 4.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [2.49200000e+03, 1.09530876e+08, 4.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [6.00000000e+03, 2.49810000e+08, 1.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       ...,
       [6.00000000e+03, 2.49810000e+08, 1.10000000e+01, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [2.49200000e+03, 1.09530876e+08, 4.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [6.00000000e+03, 2.49810000e+08, 6.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00]])

In [40]:
j_pred_gru = model_GRU.predict(I_test_bil)

NameError: name 'model_GRU' is not defined

In [153]:
mse_gru_unseen = mean_squared_error(j_test, j_pred_gru)
rmse_gru_unseen = np.sqrt(mse_gru_unseen)
print(f"Root Mean Squared Error: {rmse_gru_unseen}")
print(f"Mean Squared Error: {mse_gru_unseen}")

Root Mean Squared Error: 194.71178741722082
Mean Squared Error: 37912.68015920899


In [155]:
# Create a DataFrame of Real and Predicted values
prices_gru_unseen = pd.DataFrame({
    "Real": j_test.ravel(),
    "Predicted": j_pred_gru.ravel()
}, index = df1.index[-len(j_test): ]) 
#prices_gru_unseen.head()
prices_gru_unseen.tail()

,Real,Predicted
1048560,214,30.030535
1048561,214,30.030535
1048562,214,30.030535
1048563,238,30.895760
1048564,214,30.030535


In [46]:
y_pred_cb

array([30.00143668, 30.00312341, 28.00608967, ..., 30.00312341,
       27.99381113, 30.002073  ])

In [ ]:
## catBOOST

In [68]:
j_pred_cb = model_cb.predict(I_test_bil)

In [69]:
j_pred_cb

array([29.39583455, 29.25809166, 29.83443704, ..., 29.92551066,
       29.25809166, 29.69597849])

In [71]:
from sklearn.metrics import mean_squared_error
mse_cat_uns = mean_squared_error(j_test_bil, j_pred_cb)
rmse_cat_uns = np.sqrt(mse_cat_uns)
print(f"Root Mean Squared Error: {rmse_cat_uns}")
print(f"Mean Squared Error: {mse_cat_uns}")

Root Mean Squared Error: 195.44410908749919
Mean Squared Error: 38198.399777006285


In [54]:
dc = j_test.ravel()

In [56]:
dc

array([214, 214, 238, ..., 238, 214, 238])

In [57]:
j_pred_cb

array([30.14531898, 29.94960655, 29.14062266, ..., 28.91224665,
       29.94960655, 29.27214931])

In [58]:
cv = j_pred_cb.ravel()

In [59]:
cv

array([30.14531898, 29.94960655, 29.14062266, ..., 28.91224665,
       29.94960655, 29.27214931])

In [72]:
# Create a DataFrame of Real and Predicted values
prices_catb_unseen = pd.DataFrame({
    "Real": j_test.ravel(),
    "Predicted": j_pred_cb.ravel()
}, index = df33.index[-len(j_test): ]) 
prices_catb_unseen.head()

,Real,Predicted
801745,214,29.395835
801746,214,29.258092
801747,238,29.834437
801748,238,29.584772
801749,238,29.968514
